# RCN Inverted Recreation With Raw TangTV Input

In [ ]:
import sys
import numpy as np
import scipy
from scipy.io import readsav
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from skimage import measure
import cv2
from tqdm import tqdm
import glob
import time
import pickle 
from sklearn import metrics
import importlib

import utils.rcn_functions as rcfun; importlib.reload(rcfun)
from utils.get_file import GetTV, _find_index

In [ ]:
tv = GetTV()
files = tv.list_files()
[inverted,radii,elevation,frames,times,vid_frames,vid_times,vid] = tv.load_all(files[11])

In [ ]:
## Detect spots and save the plots for movie
rnd=np.random.permutation(120)
train_ind = rnd[:80]
valid_ind = rnd[80:]

X_train=[]
y_train=[]
for fid in tqdm(train_ind):    
    tid = _find_index(vid_times,times[fid]) #find frame id for camera image with t=times[fid]
    y_train.append(inverted[fid])
    X_train.append(vid[fid])

X_valid=[]
y_valid=[]
for fid in tqdm(valid_ind):    
    tid = _find_index(vid_times,times[fid])
    y_valid.append(inverted[fid])
    X_valid.append(vid[fid])


In [ ]:
res_size = [100, 50]
K_in =   [1000, -1]  # -1 = Full connection
K_rec = [10, 10]  # -1 = Full connection
insca =  [0.5, 1]
spra =   [0, 0.95]
bisca =  [0, 0.5]
leak =    [1, 0.1]
regu =   [0.01, 0.01]
nread = 201*201
input_dim = [240*720,201*201]

W_in = [None] * 2
W_res = [None] * 2
W_bi = [None] * 2
W_out = [None] * 2
x = [None] * 2
o = [None] * 2

In [ ]:
#### ----- Layer 1 ----- ####
layer =0

W_in[layer], W_res[layer], W_bi[layer] = rcfun.init_weights(input_dim[layer], res_size[layer], K_in[layer], K_rec[layer], insca[layer], spra[layer], bisca[layer])

t1 = time.time()  # Just to have the time
xTx = np.zeros((res_size[layer] + 1, res_size[layer] + 1), dtype='float64')
xTy = np.zeros((res_size[layer] + 1, nread), dtype='float64')
xlen = 0

print('start')
for u,d in tqdm(zip(X_train,y_train)):

    u=np.reshape(u,(1,-1))
    d=np.reshape(d,(1,-1))

    x[layer] = rcfun.res_exe(W_in[layer], W_res[layer], W_bi[layer], leak[layer], u)
    xlen += u.shape[0]
    xTx += np.dot(x[layer].T, x[layer])
    xTy += np.dot(x[layer].T, d)
t2 = time.time()
W_out[layer] = rcfun.res_train(xTx, xTy, xlen, regu[layer])

t3 = time.time()
print('Exec in %.2f - Train in %.2f sec.!' % (round(t2 - t1, 2), round(t3 - t2, 2)))
dump_data={'W_in': W_in
               ,'W_res': W_res
               ,'W_bi': W_bi
               ,'W_out': W_out
               ,'leak': leak
          }

In [ ]:
for fid in tqdm(range(170)):
    plt.clf()
    u=vid[fid]
    d=inverted[fid]
    
    u=np.reshape(u,(1,-1))
    d=np.reshape(d,(1,-1))
    
    o= np.dot(rcfun.res_exe(W_in[0], W_res[0], W_bi[0], leak[0], u),W_out[0])
    fig,axs=plt.subplots(ncols=3,nrows=1,figsize=(15,5))
    axs[0].pcolormesh(np.flipud(np.reshape(u,(240,720))))
    axs[1].pcolormesh(np.reshape(d,(201,201)))
    axs[2].pcolormesh(np.reshape(o,(201,201)),vmin=np.min(d),vmax=np.max(d))
    
    if fid in train_ind:
        axs[0].set(title='input',ylabel='Train data')
    else:
        axs[0].set(title='input',ylabel='Valid data')
    axs[1].set(title='Truth')
    axs[2].set(title='Prediction')
    outname = 'outputs/plots/%04d.png' % (fid) 
    plt.savefig(outname, bbox_inches='tight', dpi=100, facecolor='white')
    plt.close()  # Close the figure after saving


In [ ]:
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import glob

frmlist = glob.glob('outputs/plots/*.png')
# read first file to get size
first_file = frmlist[0]

fig = plt.figure()
im = plt.imshow(plt.imread(first_file), animated=True)
plt.title("RCN Model")

def update(frame):
    im.set_data(plt.imread(frmlist[frame]))
    return im,

ani = animation.FuncAnimation(fig, update, frames=len(frmlist), interval=100, blit=True)
ani.save('outputs/plots/plasmatv_surrogate.gif', writer='Pillow', fps=25)
plt.close(fig)
